## 模型一：XGB

In [1]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import xgboost as xgb

In [2]:
operation_TRAIN = pd.read_csv("../data/operation_TRAIN.csv")
transaction_TRAIN = pd.read_csv("../data/transaction_TRAIN.csv")
tag_TRAIN = pd.read_csv("../data/tag_TRAIN.csv")                                                          
operation_TRAIN = operation_TRAIN.drop_duplicates("UID", keep="last")
transaction_TRAIN = transaction_TRAIN.drop_duplicates("UID", keep="last")
train_X = pd.merge(operation_TRAIN, transaction_TRAIN, on="UID", how="outer")
train = pd.merge(train_X, tag_TRAIN, on="UID", how="left")
train = train.sort_values("UID")

pos_train = train[train["Tag"] == 1]


operation_train_new = pd.read_csv("../data/operation_train_new.csv")
transaction_train_new = pd.read_csv("../data/transaction_train_new.csv")
tag_train_new = pd.read_csv("../data/tag_train_new.csv")                                                          
operation_train_new = operation_train_new.drop_duplicates("UID", keep="last")
transaction_train_new = transaction_train_new.drop_duplicates("UID", keep="last")
train_X_new = pd.merge(operation_train_new, transaction_train_new, on="UID", how="outer")
train_new = pd.merge(train_X_new, tag_train_new, on="UID", how="left")
train_new = train_new.sort_values("UID")

train_all = pd.concat([pos_train, train_new], axis=0)
# operation_round1 = pd.read_csv("../data/operation_round1.csv")
# transaction_round1 = pd.read_csv("../data/transaction_round1.csv")                                                                                                                      
# operation_round1 = operation_round1.drop_duplicates(["UID"], keep="last")
# transaction_round1 = transaction_round1.drop_duplicates(["UID"], keep="last")                                                              
# test_C = pd.merge(operation_round1, transaction_round1, on="UID", how="outer")
# test_C = test_C.sort_values("UID")

operation_round1_new = pd.read_csv("../data/operation_round1_new.csv")
transaction_round1_new = pd.read_csv("../data/transaction_round1_new.csv")                                                                                                                      
operation_round1_new = operation_round1_new.drop_duplicates(["UID"], keep="last")
transaction_round1_new = transaction_round1_new.drop_duplicates(["UID"], keep="last")                                                              
test_C_new = pd.merge(operation_round1_new, transaction_round1_new, on="UID", how="outer")
test_C_new = test_C_new.sort_values("UID")



D:\python3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
del operation_TRAIN
del transaction_TRAIN
del tag_TRAIN
del operation_train_new
del transaction_train_new
del tag_train_new
del operation_round1_new
del transaction_round1_new

In [4]:
test_C_new["Tag"] = -1
data = pd.concat([train_all, test_C_new])
data["time_x"] = data["time_x"].fillna(value="00:00:00")
data["time_y"] = data["time_y"].fillna(value="00:00:00")
data["day_x"] = data["day_x"].fillna(value=0)
data["day_y"] = data["day_y"].fillna(value=30)
data = data.fillna(value=-99999)

data["hour_x"] = data.time_x.apply(lambda  x: x.split(":")[0])
data["minutes_x"] = data.time_x.apply(lambda  x: x.split(":")[1])
data["hour_y"] = data.time_y.apply(lambda  x: x.split(":")[0])
data["minutes_y"] = data.time_y.apply(lambda  x: x.split(":")[1])

data["day_y"] = data["day_y"] + 30
data["day_sub"] = data["day_y"] - data["day_x"]
data["hour_sub"] = (data["hour_y"].astype(int) +24) - data["hour_x"].astype(int)
data["minutes_sub"] = (data["minutes_y"].astype(int) +60) - data["minutes_x"].astype(int)

feature = data.columns.drop(["UID", "Tag"])
col_feature = ["day_x", "hour_sub", "minutes_sub", "bal", "trans_amt"]
dis_feature = feature.drop(col_feature)

for col in dis_feature:
    lbl = preprocessing.LabelEncoder()
    data[col] = lbl.fit_transform(list(data[col].values))
    
# data["time_sub"] = data["time_y"] - data["time_x"]
# data["geo_code_sub"] = data["geo_code_y"] - data["geo_code_x"]

# data["device21_x_sub"] = data["device2_x"] - data["device1_x"]
# data["device21_y_sub"] = data["device2_y"] - data["device1_y"]
# data["device1_sub"] = data["device1_y"] - data["device1_x"]
# data["device2_sub"] = data["device2_y"] - data["device2_x"]

# data["device_code21_x_sub"] = data["device_code2_x"] - data["device_code1_x"]
# data["device_code32_x_sub"] = data["device_code3_x"] - data["device_code2_x"]
# data["device_code31_x_sub"] = data["device_code3_x"] - data["device_code1_x"]
# data["device_code21_y_sub"] = data["device_code2_y"] - data["device_code1_y"]
# data["device_code32_y_sub"] = data["device_code3_y"] - data["device_code2_y"]
# data["device_code31_y_sub"] = data["device_code3_y"] - data["device_code1_y"]
# data["device_code1_sub"] = data["device_code1_y"] - data["device_code1_x"]
# data["device_code2_sub"] = data["device_code2_y"] - data["device_code2_x"]
# data["device_code3_sub"] = data["device_code3_y"] - data["device_code3_x"]

# data["mac1_sub"] = data["mac1_y"] - data["mac1_x"]
# data["mac_sub"] = data["mac2"] - data["mac1_x"]
# data["ip1_sub"] = data["ip1_y"] - data["ip1_x"]
# data["ip21_sub"] = data["ip2_sub"] - data["ip1_sub"]
# data["ip1_sub_sub"] = data["ip1_sub_y"] - data["ip1_sub_x"]
# data["acc_id21_sub"] = data["acc_id2"] - data["acc_id1"]
# data["acc_id32_sub"] = data["acc_id3"] - data["acc_id2"]
# data["acc_id31_sub"] = data["acc_id3"] - data["acc_id1"]
    


# for col in ["mode", "os", "wifi"]:
#     print("this is feature:", col)
#     onehot_feats = pd.get_dummies(data[col], prefix=col)
#     data.drop([col], axis=1, inplace=True)
#     data = pd.concat([data, onehot_feats], axis=1)
    
    
    
train = data[data["Tag"] != -1]
train_Y = train["Tag"]
train_X = train.drop(["UID", "Tag"], axis=1)
test = data[data["Tag"] == -1]
test_UID = test["UID"]
test_X = test.drop(["UID", "Tag"], axis=1)



In [5]:
params = {"booster": "gbtree",
          "objective" : "binary:logistic",
          "eval_metric" : "auc",
          "verbose" : 0, 
          "eta" : 0.01, 
          "max_delta_step" : 5, 
          "max_depth" : 20, 
          "alpha" : 10, 
          "lambda" : 10, 
          "gamma" : 2, 
          "subsample" : 0.8, 
          "colsample_bytree" : 0.8, 
          "min_child_weight" : 5, 
          "scale_pos_weight" : 0.8,
          "n_jobs" : -1}

# x_train, x_val, y_train, y_true = train_test_split(train_X, train_Y, test_size=0.2, random_state=2018)

# matrix_train = xgb.DMatrix(x_train, y_train)
# matrix_test = xgb.DMatrix(x_val)
# a_model = xgb.train(params, matrix_train, num_boost_round=100000)
# y_predict = a_model.predict(matrix_test)


# d = pd.DataFrame()
# d['prob'] = list(y_predict)
# d['y'] = list(y_true)
# d = d.sort_values(['prob'], ascending=[0])
# y = d.y
# PosAll = pd.Series(y).value_counts()[1]
# NegAll = pd.Series(y).value_counts()[0]
# pCumsum = d['y'].cumsum()
# nCumsum = np.arange(len(y)) - pCumsum + 1
# pCumsumPer = pCumsum / PosAll
# nCumsumPer = nCumsum / NegAll
# TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
# TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
# TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]

# result = 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3
# print("*******************************************************************************************************")
# print("result:", result)
# print("*******************************************************************************************************")



In [ ]:
train_matrix = xgb.DMatrix(train_X, train_Y)
test_matrix = xgb.DMatrix(test_X)

model = xgb.train(params, train_matrix, num_boost_round=50000)
predict = model.predict(test_matrix)

result = pd.DataFrame({"UID": test_UID, "Tag": predict})
result.to_csv("../submission/"+"result_time"+str(int(time.time()))+".csv", index=False)

## 模型二：LGB

In [16]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lightgbm as lgb

In [17]:
operation_train_new = pd.read_csv("../data/operation_train_new.csv")
transaction_train_new = pd.read_csv("../data/transaction_train_new.csv")
tag_train_new = pd.read_csv("../data/tag_train_new.csv")                                                          
operation_train_new = operation_train_new.drop_duplicates("UID", keep="last")
transaction_train_new = transaction_train_new.drop_duplicates("UID", keep="last")
train_X_new = pd.merge(operation_train_new, transaction_train_new, on="UID", how="outer")
train_new = pd.merge(train_X_new, tag_train_new, on="UID", how="left")
train_new = train_new.sort_values("UID")

operation_round1_new = pd.read_csv("../data/operation_round1_new.csv")
transaction_round1_new = pd.read_csv("../data/transaction_round1_new.csv")                                                                                                                      
operation_round1_new = operation_round1_new.drop_duplicates(["UID"], keep="last")
transaction_round1_new = transaction_round1_new.drop_duplicates(["UID"], keep="last")                                                              
test_C = pd.merge(operation_round1_new, transaction_round1_new, on="UID", how="outer")
test_C = test_C.sort_values("UID")

D:\python3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
del operation_train_new
del transaction_train_new
del tag_train_new
del operation_round1_new
del transaction_round1_new

In [19]:
test_C["Tag"] = -1
data = pd.concat([train_new, test_C])
data = data.drop(["time_x", "time_y"], axis=1)
feature = data.columns.drop(["UID", "Tag"])
col_feature = ["day_x", "day_y", "bal", "trans_amt"]
dis_feature = feature.drop(col_feature)

for col in dis_feature:
    lbl = preprocessing.LabelEncoder()
    data[col] = lbl.fit_transform(list(data[col].values))
    
train = data[data["Tag"] != -1]
train_Y = train["Tag"]
train_X = train.drop(["UID", "Tag"], axis=1)
test = data[data["Tag"] == -1]
test_UID = test["UID"]
test_X = test.drop(["UID", "Tag"], axis=1)



In [20]:
# def tpr_weight_funtion(y_true, y_predict):
#     d = pd.DataFrame()
#     d['prob'] = list(y_predict)
#     d['y'] = list(y_true)
#     d = d.sort_values(['prob'], ascending=[0])
#     y = d.y
#     PosAll = pd.Series(y).value_counts()[1]
#     NegAll = pd.Series(y).value_counts()[0]
#     pCumsum = d['y'].cumsum()
#     nCumsum = np.arange(len(y)) - pCumsum + 1
#     pCumsumPer = pCumsum / PosAll
#     nCumsumPer = nCumsum / NegAll
#     TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
#     TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
#     TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
     
#     return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

res = []
params = {'boosting_type': 'gbdt',
         'objective': 'binary',
         'metric': 'auc',
         'min_child_weight': 1.5,
         'num_leaves': 2 ** 5,
         'lambda_l2': 10,
         'subsample': 0.85,
         'learning_rate': 0.01,
         'seed': 2018,
         'colsample_bytree': 0.5,
         'nthread': -1}

# x_train, x_val, y_train, y_true = train_test_split(train_X, train_Y, test_size=0.2, random_state=2018)

matrix_train = lgb.Dataset(train_X, train_Y)
# matrix_test = lgb.Dataset(x_val, y_true)


model = lgb.train(params, matrix_train, num_boost_round=50000)
# res.append(tpr_weight_funtion(y_true, model.predict(x_val)))

predict = model.predict(np.array(test_X))

result = pd.DataFrame({"UID": test_UID, "Tag": predict})
result.to_csv("./"+"lgb_result_time"+str(int(time.time()))+".csv", index=False)




# print("*******************************************************************************************************")
# print("result:", result)
# print("*******************************************************************************************************")



## 模型三：XGB

In [36]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import xgboost as xgb

In [41]:
operation_TRAIN = pd.read_csv("../data/operation_TRAIN.csv")
transaction_TRAIN = pd.read_csv("../data/transaction_TRAIN.csv")
tag_TRAIN = pd.read_csv("../data/tag_TRAIN.csv")                                                          
operation_TRAIN = operation_TRAIN.drop_duplicates("UID", keep="last")
transaction_TRAIN = transaction_TRAIN.drop_duplicates("UID", keep="last")
train_X = pd.merge(operation_TRAIN, transaction_TRAIN, on="UID", how="outer")
train = pd.merge(train_X, tag_TRAIN, on="UID", how="left")
train = train.sort_values("UID")

operation_round1 = pd.read_csv("../data/operation_round1.csv")
transaction_round1 = pd.read_csv("../data/transaction_round1.csv")                                                                                                                      
operation_round1 = operation_round1.drop_duplicates(["UID"], keep="last")
transaction_round1 = transaction_round1.drop_duplicates(["UID"], keep="last")                                                              
test_C = pd.merge(operation_round1, transaction_round1, on="UID", how="outer")
test_C = test_C.sort_values("UID")

del operation_TRAIN
del transaction_TRAIN
del tag_TRAIN
del operation_round1
del transaction_round1

In [45]:
test_C["Tag"] = -1
data = pd.concat([train, test_C])
data = data.fillna({"time_x":"00:00:00", "time_y":"00:00:00"})
data = data.fillna(value=-1)
data["hour_x"] = data.time_x.apply(lambda  x: x.split(":")[0])
data["minutes_x"] = data.time_x.apply(lambda  x: x.split(":")[1])
data["hour_y"] = data.time_y.apply(lambda  x: x.split(":")[0])
data["minutes_y"] = data.time_y.apply(lambda  x: x.split(":")[1])

data = data.drop(["time_x", "time_y"], axis=1)
feature = data.columns.drop(["UID", "Tag"])
col_feature = ["day_x", "day_y", "bal", "trans_amt"]
dis_feature = feature.drop(col_feature)

for col in dis_feature:
    lbl = preprocessing.LabelEncoder()
    data[col] = lbl.fit_transform(list(data[col].values))
    
train = data[data["Tag"] != -1]
train_Y = train["Tag"]
train_X = train.drop(["UID", "Tag"], axis=1)
test = data[data["Tag"] == -1]
test_UID = test["UID"]
test_X = test.drop(["UID", "Tag"], axis=1)

C:\python3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [ ]:
params = {"booster": "gbtree",
          "objective" : "binary:logistic",
          "eval_metric" : "auc",
          "verbose" : 1, 
          "eta" : 0.01, 
          "max_delta_step" : 5, 
          "max_depth" : 20, 
          "alpha" : 10, 
          "lambda" : 10, 
          "gamma" : 2, 
          "subsample" : 0.8, 
          "colsample_bytree" : 0.8, 
          "min_child_weight" : 5, 
          "scale_pos_weight" : 0.8,
          "n_jobs" : -1}

x_train, x_val, y_train, y_true = train_test_split(train_X, train_Y, test_size=0.2, random_state=2018)

matrix_train = xgb.DMatrix(x_train, y_train)
matrix_test = xgb.DMatrix(x_val)
a_model = xgb.train(params, matrix_train, num_boost_round=3000)
y_predict = a_model.predict(matrix_test)


d = pd.DataFrame()
d['prob'] = list(y_predict)
d['y'] = list(y_true)
d = d.sort_values(['prob'], ascending=[0])
y = d.y
PosAll = pd.Series(y).value_counts()[1]
NegAll = pd.Series(y).value_counts()[0]
pCumsum = d['y'].cumsum()
nCumsum = np.arange(len(y)) - pCumsum + 1
pCumsumPer = pCumsum / PosAll
nCumsumPer = nCumsum / NegAll
TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]

result = 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3
print("*******************************************************************************************************")
print("result:", result)
print("*******************************************************************************************************")



In [ ]:
train_matrix = xgb.DMatrix(train_X, train_Y)
test_matrix = xgb.DMatrix(test_X)

model = xgb.train(params, train_matrix, num_boost_round=3000)
predict = model.predict(test_matrix)

result = pd.DataFrame({"UID": test_UID, "Tag": predict})
result.to_csv("./"+"result_time"+str(int(time.time()))+".csv", index=False)

## 模型三：lgb+lr

In [ ]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-
"""
@file: lgb+lr.py
@company: FinSight
@author: Zhao Ming
@time: 2018-10-27   14:35:40
"""

import time
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


def prepro():
    print("读取数据...")
    operation_TRAIN = pd.read_csv("../data/operation_TRAIN.csv")
    transaction_TRAIN = pd.read_csv("../data/transaction_TRAIN.csv")
    tag_TRAIN = pd.read_csv("../data/tag_TRAIN.csv")
    operation_TRAIN = operation_TRAIN.drop_duplicates("UID", keep="last")
    transaction_TRAIN = transaction_TRAIN.drop_duplicates("UID", keep="last")
    train_X = pd.merge(operation_TRAIN, transaction_TRAIN, on="UID", how="outer")
    train = pd.merge(train_X, tag_TRAIN, on="UID", how="left")
    train = train.sort_values("UID")

    operation_round1 = pd.read_csv("../data/operation_round1.csv")
    transaction_round1 = pd.read_csv("../data/transaction_round1.csv")
    operation_round1 = operation_round1.drop_duplicates(["UID"], keep="last")
    transaction_round1 = transaction_round1.drop_duplicates(["UID"], keep="last")
    test_C = pd.merge(operation_round1, transaction_round1, on="UID", how="outer")
    test_C = test_C.sort_values("UID")
    test_id = test_C.UID

    print("读取结束...")

    test_C["Tag"] = -1
    data = pd.concat([train, test_C])
    data["time_x"] = data["time_x"].fillna(value="00:00:00")
    data["time_y"] = data["time_y"].fillna(value="00:00:00")
    data["day_x"] = data["day_x"].fillna(value=0)
    data["day_y"] = data["day_y"].fillna(value=30)
    data = data.fillna(value=-99999)

    data["hour_x"] = data.time_x.apply(lambda  x: x.split(":")[0])
    data["minutes_x"] = data.time_x.apply(lambda  x: x.split(":")[1])
    data["hour_y"] = data.time_y.apply(lambda  x: x.split(":")[0])
    data["minutes_y"] = data.time_y.apply(lambda  x: x.split(":")[1])

    data["day_y"] = data["day_y"] + 30
    data["day_sub"] = data["day_y"] - data["day_x"]
    data["hour_sub"] = (data["hour_y"].astype(int) +24) - data["hour_x"].astype(int)
    data["minutes_sub"] = (data["minutes_y"].astype(int) +60) - data["minutes_x"].astype(int)

    feature = data.columns.drop(["UID", "Tag"])
    col_feature = ["day_x", "hour_sub", "minutes_sub", "bal", "trans_amt"]
    dis_feature = feature.drop(col_feature)

    return data, col_feature, dis_feature, test_id


def gbdt_lr_predict(data, dis_feature, con_feature, test_id):
    """离散特征one-hot"""
    print("开始one-hot...")
    for col in dis_feature:
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(list(data[col].values))
    print("one-hot结束")

    train = data[data["Tag"] != -1]
    target = train.pop("Tag")
    test = data[data["Tag"] == -1]
    test.drop(["Tag"], axis=1, inplace=True)

    print("划分数据集...")
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2018)

    print("开始训练gbdt...")
    gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                            objective='binary',
                            metric= 'auc',
                            min_child_weight= 1.5,
                            num_leaves= 2 ** 5,
                            lambda_l2= 10,
                            subsample=0.85,
                            learning_rate= 0.01,
                            seed=2018,
                            colsample_bytree=0.5,
                            n_estimators=2000,
                            n_jobs= -1)
    gbm.fit(x_train, y_train,
            eval_set=[(x_train, y_train), (x_val, y_val)],
            eval_names=["train", "val"],
            eval_metric="binary_logloss")
    model = gbm.booster_
    print("训练得到叶子数...")
    gbdt_feats_train = model.predict(train, pred_leaf=True)
    gbdt_feats_test = model.predict(test, pred_leaf=True)
    gbdt_feats_name = ["gbdt_leaf_" + str(i) for i in range(gbdt_feats_train.shape[1])]
    df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns=gbdt_feats_name)
    df_test_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns=gbdt_feats_name)

    print("构造新的数据集...")
    train = pd.concat([train, df_train_gbdt_feats], axis=1)
    test = pd.concat([test, df_test_gbdt_feats], axis=1)
    train_len = train.shape[0]
    data = pd.concat([train, test])
    del train
    del test
    gc.collect()

    """连续特征归一化"""
    print("开始归一化...")
    scaler = MinMaxScaler()
    for col in con_feature:
        data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1))
    print("归一化结束")

    """叶子数One-Hot"""
    print("开始one-hot...")
    for col in gbdt_feats_name:
        print("this is feature:", col)
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(list(data[col].values))
    print("one-hot结束")

    train = data[:train_len]
    test = data[train_len:]
    del data
    gc.collect()

    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state=2018)
    print("开始训练lr...")
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])
    print("tr-logloss:", tr_logloss)
    val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
    print("val-logloss:", val_logloss)

    fpr, tpr, threshold = roc_curve(y_val, lr.predict_proba(x_val)[:, 1])
    roc_auc = auc(fpr, tpr)
    lw = 2
    plt.plot(fpr, tpr, color="darkorange", lw=lw, label="ROC curve (area = %0.5f)" % roc_auc)
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("gbdt_lr" + "ROC")
    plt.legend(loc="lower right")
    plt.show()

    print("开始预测...")
    y_pred = lr.predict_proba(test)[:, 1]
    print("写入结果...")
    submission = pd.DataFrame({"UID": test_id, "Tag": y_pred})
    submission.to_csv("../submission/"+str(int(time.time()))+".csv", index=False)
    print("结束")


if __name__ == "__main__":
    print("time start:\n")
    print(time.clock() / 60)

    data, col_feature, dis_feature, test_id = prepro()
    gbdt_lr_predict(data, dis_feature, col_feature, test_id)

    print('time end')
    print(time.clock() / 60)





## 模型四：xgb+lr

In [ ]:
# !/usr/bin/python3
# -*- coding: utf-8 -*-
"""
@file: lgb+lr.py
@company: FinSight
@author: Zhao Ming
@time: 2018-11-01   15:20:10
"""

import time
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


def prepro():
    print("读取数据...")
    operation_TRAIN = pd.read_csv("../data/operation_TRAIN.csv")
    transaction_TRAIN = pd.read_csv("../data/transaction_TRAIN.csv")
    tag_TRAIN = pd.read_csv("../data/tag_TRAIN.csv")
    operation_TRAIN = operation_TRAIN.drop_duplicates("UID", keep="last")
    transaction_TRAIN = transaction_TRAIN.drop_duplicates("UID", keep="last")
    train_X = pd.merge(operation_TRAIN, transaction_TRAIN, on="UID", how="outer")
    train = pd.merge(train_X, tag_TRAIN, on="UID", how="left")
    train = train.sort_values("UID")

    operation_round1 = pd.read_csv("../data/operation_round1.csv")
    transaction_round1 = pd.read_csv("../data/transaction_round1.csv")
    operation_round1 = operation_round1.drop_duplicates(["UID"], keep="last")
    transaction_round1 = transaction_round1.drop_duplicates(["UID"], keep="last")
    test_C = pd.merge(operation_round1, transaction_round1, on="UID", how="outer")
    test_C = test_C.sort_values("UID")
    test_id = test_C.UID

    print("读取结束...")

    test_C["Tag"] = -1
    data = pd.concat([train, test_C])
    data["time_x"] = data["time_x"].fillna(value="00:00:00")
    data["time_y"] = data["time_y"].fillna(value="00:00:00")
    data["day_x"] = data["day_x"].fillna(value=0)
    data["day_y"] = data["day_y"].fillna(value=30)
    data = data.fillna(value=-99999)

    data["hour_x"] = data.time_x.apply(lambda  x: x.split(":")[0])
    data["minutes_x"] = data.time_x.apply(lambda  x: x.split(":")[1])
    data["hour_y"] = data.time_y.apply(lambda  x: x.split(":")[0])
    data["minutes_y"] = data.time_y.apply(lambda  x: x.split(":")[1])

    data["day_y"] = data["day_y"] + 30
    data["day_sub"] = data["day_y"] - data["day_x"]
    data["hour_sub"] = (data["hour_y"].astype(int) +24) - data["hour_x"].astype(int)
    data["minutes_sub"] = (data["minutes_y"].astype(int) +60) - data["minutes_x"].astype(int)

    feature = data.columns.drop(["UID", "Tag"])
    col_feature = ["day_x", "hour_sub", "minutes_sub", "bal", "trans_amt"]
    dis_feature = feature.drop(col_feature)

    return data, col_feature, dis_feature, test_id


def gbdt_lr_predict(data, dis_feature, con_feature, test_id):
    """离散特征one-hot"""
    print("开始one-hot...")
    for col in dis_feature:
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(list(data[col].values))
    print("one-hot结束")

    train = data[data["Tag"] != -1]
    target = train.pop("Tag")
    test = data[data["Tag"] == -1]
    test.drop(["Tag"], axis=1, inplace=True)

    print("划分数据集...")
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2018)

    print("开始训练gbdt...")

    # lgb模型
    # lgbm = lgb.LGBMClassifier(booster="gbtree",
    #                         subsample=0.8,
    #                         min_child_weight=0.2,
    #                         colsample_bytree=0.8,
    #                         num_leaves=100,
    #                         max_depth=20,
    #                         reg_lambda=20,
    #                         reg_alpha=10,
    #                         learning_rate=0.01,
    #                         n_estimators=2000,
    #                         n_jobs=-1)
    # lgbm.fit(x_train, y_train,
    #         eval_set=[(x_train, y_train), (x_val, y_val)],
    #         eval_names=["train", "val"],
    #         eval_metric="binary_logloss")
    # model = lgbm.booster_
    
    # xgb模型
    params = {"booster": "gbtree",
              "objective": "binary:logistic",
              "eval_metric": "auc",
              "verbose": 1,
              "eta": 0.01,
              "max_delta_step": 5,
              "max_depth": 20,
              "alpha": 10,
              "lambda": 10,
              "gamma": 2,
              "subsample": 0.8,
              "colsample_bytree": 0.8,
              "min_child_weight": 5,
              "scale_pos_weight": 0.8,
              "n_jobs": -1}
    matrix_train = xgb.DMatrix(x_train, y_train)
    Matrix_Train = xgb.DMatrix(train, target)
#     matrix_val = xgb.DMatrix(x_val)
    matrix_test = xgb.DMatrix(test)
    model = xgb.train(params, matrix_train, num_boost_round=3000)


    print("训练得到叶子数...")
    gbdt_feats_train = model.predict(Matrix_Train, pred_leaf=True)
    gbdt_feats_test = model.predict(matrix_test, pred_leaf=True)
    gbdt_feats_name = ["gbdt_leaf_" + str(i) for i in range(gbdt_feats_train.shape[1])]
    df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns=gbdt_feats_name)
    df_test_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns=gbdt_feats_name)

    print("构造新的数据集...")
    train = pd.concat([train, df_train_gbdt_feats], axis=1)
    test = pd.concat([test, df_test_gbdt_feats], axis=1)
    train_len = train.shape[0]
    data = pd.concat([train, test])
    del train
    del test
    gc.collect()

    """连续特征归一化"""
    print("开始归一化...")
    scaler = MinMaxScaler()
    for col in con_feature:
        data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1))
    print("归一化结束")

    """叶子数One-Hot"""
    print("开始one-hot...")
    for col in gbdt_feats_name:
        print("this is feature:", col)
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(list(data[col].values))
    print("one-hot结束")

    train = data[:train_len]
    test = data[train_len:]
    del data
    gc.collect()

    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state=2018)
    print("开始训练lr...")
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])
    print("tr-logloss:", tr_logloss)
    val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
    print("val-logloss:", val_logloss)

    fpr, tpr, threshold = roc_curve(y_val, lr.predict_proba(x_val)[:, 1])
    roc_auc = auc(fpr, tpr)
    lw = 2
    plt.plot(fpr, tpr, color="darkorange", lw=lw, label="ROC curve (area = %0.5f)" % roc_auc)
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("gbdt_lr" + "ROC")
    plt.legend(loc="lower right")
    plt.show()

    print("开始预测...")
    y_pred = lr.predict_proba(test)[:, 1]
    print("写入结果...")
    submission = pd.DataFrame({"UID": test_id, "Tag": y_pred})
    submission.to_csv("../submission/"+str(int(time.time()))+".csv", index=False)
    print("结束")


if __name__ == "__main__":
    print("time start:\n")
    print(time.clock() / 60)

    data, col_feature, dis_feature, test_id = prepro()
    gbdt_lr_predict(data, dis_feature, col_feature, test_id)

    print('time end')
    print(time.clock() / 60)



